In [1]:
import findspark
findspark.init('/home/ubuntu/spark-3.3.1-bin-hadoop3')

from pyspark.sql import SparkSession

from pyspark.sql.functions import col, expr
from pyspark.sql.types import StructField, StructType, StringType
from delta.tables import DeltaTable
from pyspark.sql import Row
from datetime import datetime, date
import os
import os.path
import pandas as pd

spark = SparkSession.builder \
    .enableHiveSupport() \
    .appName("pyspark_project_2.2") \
    .master("local[*]") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.2.1") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
    .getOrCreate()

:: loading settings :: url = jar:file:/home/ubuntu/spark-3.3.1-bin-hadoop3/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-40ad792d-92e6-4f08-9330-f01ee51a3f50;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.2.1 in central
	found io.delta#delta-storage;1.2.1 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 293ms :: artifacts dl 7ms
	:: modules in use:
	io.delta#delta-core_2.12;1.2.1 from central in [default]
	io.delta#delta-storage;1.2.1 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evi

23/02/07 14:56:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


0. Подготовительные действия (Запись исходного зеркала в формате дельта; Создание логов)

In [2]:
first_delta = "/home/ubuntu/project_2.2/data_deltas/1000" # путь к 1-й дельте
path_to_mirror = "/home/ubuntu/project_2.2/mirr_md_account_d/" # путь для сохранения зеркала в формате дельта

#Запись исходного зеркала в формате дельа
spark.read.format("csv")\
    .option("header",True)\
    .option("sep",";")\
    .option("inferSchema",True)\
    .load(first_delta)\
    .write.format("delta")\
    .mode('overwrite')\
    .save(path_to_mirror+"deltaTable")

deltaTable = DeltaTable.forPath(spark, path_to_mirror+"deltaTable")
deltaTable.toDF().show(30,False,True)

-RECORD 0------------------------------------
 DATA_ACTUAL_DATE     | 01.01.2018           
 DATA_ACTUAL_END_DATE | 31.12.2050           
 ACCOUNT_RK           | 36237725             
 ACCOUNT_NUMBER       | 30425840700000583001 
 CHAR_TYPE            | A                    
 CURRENCY_RK          | 35                   
 CURRENCY_CODE        | 840                  
 CLIENT_ID            | null                 
 BRANCH_ID            | 101                  
 OPEN_IN_INTERNET     | null                 
-RECORD 1------------------------------------
 DATA_ACTUAL_DATE     | 01.01.2018           
 DATA_ACTUAL_END_DATE | 31.12.2050           
 ACCOUNT_RK           | 24656                
 ACCOUNT_NUMBER       | 30114840700000770002 
 CHAR_TYPE            | A                    
 CURRENCY_RK          | 35                   
 CURRENCY_CODE        | 840                  
 CLIENT_ID            | 2                    
 BRANCH_ID            | 107                  
 OPEN_IN_INTERNET     | null      

In [3]:
path_to_logs = "/home/ubuntu/project_2.2/logs/logs.csv" # путь к файлу с логами

#Создание датафрейма для логов и сохранение в формате csv 
logs = pd.DataFrame(columns = ['start_load', 'end_load', 'delta_id', 'table_name'])
logs = logs.to_csv(path_to_logs, sep=";", header=True, index = True, index_label = str('action_id'))
logs = pd.read_csv(path_to_logs, sep=";", index_col='action_id')
logs.head()

,start_load,end_load,delta_id,table_name
action_id,,,,


1. Функция для обновления зеркала

In [4]:
def mirror_update (path_to_deltas, unique_key, table_name):
      
    global logs
    
    list_delta_id = sorted(os.listdir(path_to_deltas)) #Получаем сортированнный список id дельт
    delta_id = list_delta_id [0] #Первая дельта в списке
    last_delta = list_delta_id [-1] # Последняя дельта в списке
    
    for delta_id in list_delta_id:
        
        logs = pd.read_csv(path_to_logs, sep=";", index_col='action_id')
        
        while str(delta_id) in str(logs['delta_id']):              #Пока id дельты есть в логах
            delta_id = int(delta_id) + 1                           #берем следующий id дельты,
            if delta_id > int(last_delta):                         #если он больше последнего существующего в директории,
                return print('All deltas already processed')       #выполенение функции прерывается
        else:
            path_to_delta=path_to_deltas + str(delta_id)           #Если id дельты в логах нет, возвращается путь к ней
            print('Delta', delta_id, 'in progress')

        start_load = datetime.now()

        df_delta = spark.read.format("csv")\
           .option("header",True)\
           .option("sep",";")\
           .option("inferSchema",True)\
           .load(path_to_delta)

        deltaTable.alias("mir").merge(
        source = df_delta.alias("df_delta"),
        condition = ''.join(["mir."+unique_key[i]+" = df_delta."+\
                unique_key[i]+" and "\
                for i in range(len(unique_key))]))\
            .whenMatchedUpdateAll()\
            .whenNotMatchedInsertAll()\
            .execute()
        
        deltaTable.toDF().write.format("csv")\
            .option("sep",";")\
            .option("header",True)\
            .option("inferSchema",True)\
            .mode('overwrite')\
            .save(path_to_mirror+table_name)
        
        end_load = datetime.now()

        logs = logs.append({'start_load':str(start_load), 'end_load':str(end_load), 'delta_id':str(delta_id), \
                            'table_name':str(table_name)},ignore_index=True)
    
        logs = logs.to_csv(path_to_logs, sep=";", header=True, index = True, index_label = str('action_id'))

3. Объявляем переменные (путь к дельтам, наименование таблицы, уникальный ключ) и вызываем функцию для обработки всех дельт

In [5]:
path_to_deltas = "/home/ubuntu/project_2.2/data_deltas/"
unique_key = ["account_rk"]
table_name = 'md_account_d.csv'

mirror_update (path_to_deltas, unique_key, table_name)

Delta 1000 in progress
23/02/07 15:06:27 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


/tmp/ipykernel_4898/3314074539.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  logs = logs.append({'start_load':str(start_load), 'end_load':str(end_load), 'delta_id':str(delta_id), \


Delta 1001 in progress


/tmp/ipykernel_4898/3314074539.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  logs = logs.append({'start_load':str(start_load), 'end_load':str(end_load), 'delta_id':str(delta_id), \


Delta 1002 in progress


/tmp/ipykernel_4898/3314074539.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  logs = logs.append({'start_load':str(start_load), 'end_load':str(end_load), 'delta_id':str(delta_id), \


Delta 1003 in progress


/tmp/ipykernel_4898/3314074539.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  logs = logs.append({'start_load':str(start_load), 'end_load':str(end_load), 'delta_id':str(delta_id), \


4. Конечное состояние зеркала, логи.

In [6]:
#Конечное состояние зеркала
spark.read.format("csv")\
    .option("sep",";")\
    .option("header",True)\
    .option("inferSchema",True)\
    .load(path_to_mirror+table_name)\
    .show(30, False, True)

-RECORD 0------------------------------------
 DATA_ACTUAL_DATE     | 15.02.2018           
 DATA_ACTUAL_END_DATE | 31.12.2050           
 ACCOUNT_RK           | 13560                
 ACCOUNT_NUMBER       | 30110810300000008001 
 CHAR_TYPE            | A                    
 CURRENCY_RK          | 34                   
 CURRENCY_CODE        | 643                  
 CLIENT_ID            | 20                   
 BRANCH_ID            | 105                  
 OPEN_IN_INTERNET     | Y                    
-RECORD 1------------------------------------
 DATA_ACTUAL_DATE     | 21.04.2018           
 DATA_ACTUAL_END_DATE | 31.12.2050           
 ACCOUNT_RK           | 13630                
 ACCOUNT_NUMBER       | 30102810900000002185 
 CHAR_TYPE            | A                    
 CURRENCY_RK          | 34                   
 CURRENCY_CODE        | 643                  
 CLIENT_ID            | 21                   
 BRANCH_ID            | 107                  
 OPEN_IN_INTERNET     | null      

In [7]:
logs = pd.read_csv(path_to_logs, sep=";", index_col='action_id')
logs.head()

,start_load,end_load,delta_id,table_name
action_id,,,,
0,2023-02-07 15:06:24.669704,2023-02-07 15:06:30.905037,1000,md_account_d.csv
1,2023-02-07 15:06:30.918851,2023-02-07 15:06:34.712864,1001,md_account_d.csv
2,2023-02-07 15:06:34.720462,2023-02-07 15:06:38.008155,1002,md_account_d.csv
3,2023-02-07 15:06:38.014463,2023-02-07 15:06:41.052566,1003,md_account_d.csv


In [8]:
deltaTable.history().show(10,False,True)

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 version             | 4                                                                                                                                                                                                                                                                                 
 timestamp           | 2023-02-07 15:06:39.038                                                                                                                                                                                                                                                           
 userId              | null                                                                               

5. Создание новой дельты.

In [9]:
#Запись в таблице с account_rk = 18165 до обработки дельты 1004
df = spark.read.format("csv")\
    .option("sep",";")\
    .option("header",True)\
    .option("inferSchema",True)\
    .load(path_to_mirror+table_name)
df = df.select("*").filter(df.ACCOUNT_RK == 18165).show(10, False,True)

-RECORD 0------------------------------------
 DATA_ACTUAL_DATE     | 21.04.2018           
 DATA_ACTUAL_END_DATE | 31.12.2050           
 ACCOUNT_RK           | 18165                
 ACCOUNT_NUMBER       | 30232978900450001001 
 CHAR_TYPE            | P                    
 CURRENCY_RK          | 44                   
 CURRENCY_CODE        | 978                  
 CLIENT_ID            | 13                   
 BRANCH_ID            | 120                  
 OPEN_IN_INTERNET     | Y                    



In [10]:
mirror_update (path_to_deltas, unique_key, table_name)

Delta 1004 in progress


All deltas already processed


/tmp/ipykernel_4898/3314074539.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  logs = logs.append({'start_load':str(start_load), 'end_load':str(end_load), 'delta_id':str(delta_id), \


In [11]:
#Запись в таблице с account_rk = 18165 после обработки дельты 1004
df = spark.read.format("csv")\
    .option("sep",";")\
    .option("header",True)\
    .option("inferSchema",True)\
    .load(path_to_mirror+table_name)
df = df.select("*").filter(df.ACCOUNT_RK == 18165).show(10, False,True)

-RECORD 0------------------------------------
 DATA_ACTUAL_DATE     | 21.04.2018           
 DATA_ACTUAL_END_DATE | 31.12.2050           
 ACCOUNT_RK           | 18165                
 ACCOUNT_NUMBER       | 30232978900450001001 
 CHAR_TYPE            | P                    
 CURRENCY_RK          | 44                   
 CURRENCY_CODE        | 978                  
 CLIENT_ID            | 1                    
 BRANCH_ID            | 111                  
 OPEN_IN_INTERNET     | N                    

